# Theme 7: Homogeneous Coordinates and Camera Calibration

**Objective:** In this exercise, we learn to use homogeneous coordinates for geometric operations and perform camera calibration using OpenCV.

**Instructions:**
1.  Read the theoretical explanations for each question.
2.  Run the code blocks to see the worked solutions.
3.  For Question 6 (Practical), follow the instructions to calibrate your own camera.


## Question 1: Line Intersection using Homogeneous Coordinates

**Task:** Calculate the intersection of lines $2x - y + 3 = 0$ and $3x + y - 1 = 0$ in the homogeneous coordinate system.

**Theory:**
In homogeneous coordinates, a 2D line $ax + by + c = 0$ is represented by the vector **l** = $[a, b, c]^T$.
The intersection of two lines **l1** and **l2** is the point **x** given by the cross product:
$$ \mathbf{x} = \mathbf{l}_1 	imes \mathbf{l}_2 $$
The result is a homogeneous point $[x', y', w']^T$. To get Cartesian coordinates $(x, y)$, we normalize: $x = x'/w', y = y'/w'$.


In [ ]:
import numpy as np

# Line 1: 2x - y + 3 = 0 -> [2, -1, 3]
l1 = np.array([2, -1, 3])

# Line 2: 3x + y - 1 = 0 -> [3, 1, -1]
l2 = np.array([3, 1, -1])

# Intersection is the cross product
point_hom = np.cross(l1, l2)

print(f"Homogeneous Intersection Point: {point_hom}")

# Normalize to Cartesian (divide by w)
if point_hom[2] != 0:
    point_cartesian = point_hom[:2] / point_hom[2]
    print(f"Cartesian Coordinates (x, y): {point_cartesian}")
else:
    print("Point is at infinity.")


## Question 2: Point at Infinity

**Task:** Which of the following 2D coordinates (in homogeneous format) is a point at infinity?
1. $[3, 3, -2]^T$
2. $[10, -2, 0]^T$
3. $[0, 0, 0]^T$

**Answer:**
*   **$[10, -2, 0]^T$** is the point at infinity.
*   **Reasoning:** In homogeneous coordinates $[x, y, w]^T$, points with $w=0$ represent points at infinity (direction vectors). $w 
eq 0$ represents finite points. $[0,0,0]^T$ is not a valid homogeneous coordinate.


## Question 3: Camera Intrinsic Matrix Analysis

**Task:** Consider the following camera intrinsic matrix $K$:
$$ K = \begin{bmatrix} 2000 & 0 & 960 \\ 0 & 2000 & 540 \\ 0 & 0 & 1 \end{bmatrix} $$

**Analysis:**
The intrinsic matrix usually follows the form:
$$ K = \begin{bmatrix} f_x & s & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix} $$
where:
*   $f_x, f_y$: Focal lengths in pixels.
*   $c_x, c_y$: Principal point (optical center) in pixels.
*   $s$: Skew coefficient (usually 0).

**Answers:**
1.  **Camera Model:** Pinhole Camera Model (with square pixels, as $f_x = f_y$).
2.  **Focal Length ($f$):** 2000 pixels.
3.  **Center of Projection ($c_x, c_y$):** (960, 540).


In [ ]:
K = np.array([
    [2000, 0, 960],
    [0, 2000, 540],
    [0, 0, 1]
])

print("Intrinsic Matrix K:\n", K)
print(f"Focal Length (fx, fy): {K[0,0]}, {K[1,1]}")
print(f"Principal Point (cx, cy): {K[0,2]}, {K[1,2]}")


## Question 4: 3D Point Projection

**Task:** Calculate the image coordinates of the real-world point $\mathbf{X} = [1000, -500, 1500]^T$ using the camera matrix $K$ from Q3.

**Theory:**
The projection equation is $\lambda \mathbf{x} = K \mathbf{X}$, where $\mathbf{X}$ is the 3D point (Cartesian) and $\mathbf{x}$ is the homogeneous image point.
1.  Convert $\mathbf{X}$ to homogeneous 3D coordinates (append 1? Or just multiply $K$ by $[X, Y, Z]^T$ if we assume extrinsic matrix is Identity $[I | 0]$).
    *   *Note:* The question implies the point is in the **camera coordinate frame** already.
    *   Equation: $\mathbf{x}_{hom} = K [X, Y, Z]^T$.
2.  Normalize to get pixel coordinates $u, v$.


In [ ]:
# Real-world point in camera coordinates
X_world = np.array([1000, -500, 1500])

# Project: x_hom = K * X
# Since K is 3x3 and X is 3x1, we perform matrix multiplication directly.
x_hom = K @ X_world

print(f"Projected Homogeneous Point: {x_hom}")

# Normalize (u = x'/z', v = y'/z')
x_image = x_hom[:2] / x_hom[2]
print(f"Image Coordinates (u, v): {x_image}")


## Question 5: Radial Distortion Correction

**Task:** Perform radial distortion correction for image point $\mathbf{x}_d = [200, 300]$ with distortion parameters $r_1 = 0.2, r_2 = -0.7$.

**Note:** The phrasing "correction for image point" combined with the parameters typically implies applying the correction model to recover the undistorted point, or conversely, applying distortion. Given the solved examples in this course context, we will apply the polynomial scaling factor to the normalized coordinates.
Model used: $\mathbf{x}_{corr} = \mathbf{x}_{norm} \cdot (1 + r_1 ho^2 + r_2 ho^4)$, then denormalize.

**Steps:**
1.  Normalize the pixel coordinate $(u, v)$ to intrinsic coordinates $(x, y)$ using $K$.
    $$ x = (u - c_x) / f_x, \quad y = (v - c_y) / f_y $$
2.  Calculate radius squared $r^2 = x^2 + y^2$.
3.  Calculate distortion factor $D = 1 + r_1 r^2 + r_2 r^4$.
4.  Apply correction: $x' = x \cdot D, \quad y' = y \cdot D$.
5.  Convert back to pixel coordinates.


In [ ]:
xd = np.array([200, 300]) # Distorted pixel coordinates
fx, fy = 2000, 2000
cx, cy = 960, 540

# 1. Normalize
x_norm = (xd[0] - cx) / fx
y_norm = (xd[1] - cy) / fy

# 2. Radius squared
rho2 = x_norm**2 + y_norm**2
rho4 = rho2**2

# Distortion coefficients (r1, r2)
r1, r2 = 0.2, -0.7

# 3. Distortion Factor
D = 1 + r1 * rho2 + r2 * rho4

# 4. Corrected Normalized Coordinates
x_corr_norm = x_norm * D
y_corr_norm = y_norm * D

# 5. Back to Pixel Coordinates
u_corr = x_corr_norm * fx + cx
v_corr = y_corr_norm * fy + cy

print(f"Original Distorted Point: {xd}")
print(f"Radial Distance (rho): {np.sqrt(rho2):.4f}")
print(f"Distortion Factor: {D:.4f}")
print(f"Corrected Point: [{u_corr:.2f}, {v_corr:.2f}]")


## Question 6: Camera Calibration (Practical)

**Task:** Calibrate the camera of your personal phone.

**Instructions:**
1.  Open `calibration-pattern.pdf` (from Moodle) on your monitor full-screen.
2.  Take **three** photographs of the pattern from different angles (keep distance roughly constant).
3.  Upload the images to this Colab/Notebook directory as `calib1.jpg`, `calib2.jpg`, `calib3.jpg`.
4.  Run the code below.
5.  (Optional) Calculate focal length in mm by replacing `pixel_density_mm`.

**Code:**


In [ ]:
import cv2 as cv
import glob

def calibrate_my_camera():
    # 1. Setup Patterns
    pattern_size = (9, 6) # Inner corners
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

    objpoints = [] # 3D points
    imgpoints = [] # 2D points

    # 2. Load Images
    images = glob.glob('calib*.jpg')
    print(f"Found {len(images)} images: {images}")

    if not images:
        print("No images found! Please upload 'calib1.jpg', 'calib2.jpg', etc.")
        return

    gray = None
    for fname in images:
        img = cv.imread(fname)
        if img is None: continue
        
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # Find corners
        ret, corners = cv.findChessboardCorners(gray, pattern_size, None)
        
        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
            print(f"Pattern found in {fname}")
            # cv.drawChessboardCorners(img, pattern_size, corners, ret) # Optional visualization
        else:
            print(f"Pattern NOT found in {fname}")

    if not objpoints:
        print("Calibration failed: No patterns detected.")
        return

    # 3. Calibrate
    # Use the shape of the last valid gray image
    ret, K, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    print("\n=== Calibration Results ===")
    print(f"RMS Error: {ret:.4f}")
    print("\nCamera Matrix (K):\n", K)
    print("\nDistortion Coefficients:\n", dist.ravel())

    # 4. Focal Length in mm
    # Find your phone's pixel density (pixels per mm) online.
    # Example: 400 PPI / 25.4 = ~15.7 pixels/mm (This varies widely!)
    # Default value below is arbitrary placeholder from exercise text.
    pixel_density_mm = 712.76 
    focal_length_mm = K[0, 0] / pixel_density_mm
    print(f"\nEstimated Focal Length: {focal_length_mm:.2f} mm (assuming density={pixel_density_mm})")

# Run the calibration function
calibrate_my_camera()
